In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
import torchvision
from torchvision import datasets,transforms,models

from sklearn.model_selection import train_test_split

import help
from MyDataSet import MyDataset
from medicalnet_model import generate_model

F:\Python\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
F:\Python\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
X_train, X_val, y_train, y_val = train_test_split(help.features,help.labels['label'].values,test_size = 0.2,random_state = 42,stratify=help.labels['label'].values)

In [ ]:
train_datasets = MyDataset(datas=X_train,labels=y_train,shape=3,input_D=help.input_D,input_H=help.input_H,input_W=help.input_W,phase='train')
val_datasets = MyDataset(datas=X_val,labels=y_val,shape=3,input_D=help.input_D,input_H=help.input_H,input_W=help.input_W,phase='train')
#test_datasets = help.MyDataset(images=X_test,labels=y_test,shape=3,input_D=help.input_D,input_H=help.input_H,input_W=help.input_W)

train_loader = torch.utils.data.DataLoader(dataset=train_datasets, batch_size=2, shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset=val_datasets, batch_size=1, shuffle=False)

In [ ]:
checkpoint = torch.load(help.checkpoint_pretrain_resnet_10_23dataset,map_location=help.device)

In [ ]:
medicanet_resnet3d_10,parameters = generate_model(sample_input_W=help.input_W,
                                                   sample_input_H=help.input_H,
                                                   sample_input_D=help.input_D,
                                                   num_seg_classes=help.num_seg_classes,
                                                   phase='train',
                                                   pretrain_path=help.checkpoint_pretrain_resnet_10_23dataset)
params = [
        { 'params': parameters['base_parameters'], 'lr': sets.learning_rate }, 
        { 'params': parameters['new_parameters'], 'lr': sets.learning_rate*100 }
        ]
optimizer = optim.Adam(params, weight_decay=1e-3)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

In [ ]:
epochs = 100
help.train_data(medicanet_resnet3d_10,train_loader,val_loader,epochs,optimizer,scheduler,help.criterion,help.medicanet_3d_resnet10_checkpoint_path,help.device)

In [ ]:
medicanet_resnet3d_10 = generate_model(sample_input_W=help.input_W,
                                       sample_input_H=help.input_H,
                                       sample_input_D=help.input_D,
                                       num_seg_classes=help.num_seg_classes,
                                       phase='train',
                                       pretrain_path=help.checkpoint_pretrain_resnet_10_23dataset)
model_dict = medicanet_resnet3d_10.state_dict()
####去除不属于model_dict的键值
pretrained_dict={ k : v for k, v in checkpoint.get('state_dict').items() if k in model_dict.keys()}
###更新现有的model_dict的值
model_dict.update(pretrained_dict)
medicanet_resnet3d_10.load_state_dict(model_dict)
    
for param in medicanet_resnet3d_10.parameters():
    param.requires_grad = False
for layer in [medicanet_resnet3d_10.avgpool,medicanet_resnet3d_10.fc]:
    for name, value in layer.named_parameters():
        value.requires_grad = True

In [ ]:
epochs = 100
medicanet_3d_resnet10_checkpoint_path = 'medicanet_3d_resnet10_checkpoint}.pth'.format(ii)
params = [{'params':medicanet_resnet3d_10.avgpool.parameters()},{'params':medicanet_resnet3d_10.fc.parameters()}]
optimizer = optim.SGD(params,momentum=0.9, lr=0.001, weight_decay=1e-3)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
help.train_data(medicanet_resnet3d_10,train_loader,val_loader,epochs,optimizer,scheduler,help.print_every,help.criterion,medicanet_3d_resnet10_checkpoint_path,help.device)


In [ ]:
test_datasets = MyDataset(datas=help.temp_data,shape=3,input_D=help.input_D,input_H=help.input_H,input_W=help.input_W,phase='test')
test_loader = DataLoader(dataset=test_datasets)

In [ ]:
loadmodel = help.load_checkpoint(help.medicanet_3d_resnet10_checkpoint_path,'medicanet_resnet3d_10','test',help.device)
help.all_predict(test_loader,loadmodel,help.device,help.result_medicanet_3d_resnet10)